In [ ]:
from dotenv import load_dotenv
import gradio as gr
from langchain_groq import ChatGroq

from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
MODEL = 'openai/gpt-oss-20b'
DB_Name = "vector_db"
load_dotenv(override=True)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_Name, embedding_function=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
llm = ChatGroq(temperature=0, model_name=MODEL) 

In [ ]:
retriever.invoke("Who is Avery")

In [ ]:
llm.invoke("Who is Avery")

In [ ]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [ ]:
def answer_question(question, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [ ]:
gr.ChatInterface(fn=answer_question).launch()